In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np

In [ ]:
podatkovna_baza_kart = pd.read_csv("specificne_lastnosti_kart.csv")
pojavitve_barve = podatkovna_baza_kart.groupby("barva").size()
verjetnosti_barve = pojavitve_barve / len(podatkovna_baza_kart)

verjetnosti_barve

In [ ]:
def odstrani_konce(beseda, konci):
    for konec in konci:
        if beseda.endswith(konec):
            return beseda[:-len(konec)]
    return beseda

# Tukaj bi bilo morda vredno poizkusiti brez korenov besed. Namreč v resnici pri magicu ni tako zelo veliko besed, zato korenjenje morda sploh ni potrebno
# Naredimo lahko funkcijo s koreni in brez

konci_ki_jih_bomo_odstranili = ["ing", "ed", "en", "s"]
 
def koren_besede(beseda, ne_naredi_korenov = False):
    beseda = ''.join(znak for znak in beseda if znak.isalpha()) #  
    if not beseda:
        return '$'
    if not ne_naredi_korenov:
        beseda = odstrani_konce(beseda, konci_ki_jih_bomo_odstranili)
        beseda = beseda.rstrip('aeiou')
    if not beseda:
        return '@'
    return beseda

def koreni_besed(niz, ne_naredi_korenov = False):
    return pd.Series(sorted({
        koren_besede(beseda, ne_naredi_korenov=ne_naredi_korenov) for beseda in str(niz).replace(')(', ') (').replace('-', ' ').lower().split() if beseda
    }))

In [ ]:
opis = """Whenever another creature enters the battlefield under your control, you gain life equal to that creature's toughness.
(1)(g)(w), (tap): Populate. (Put a token onto the battlefield that's a copy of a creature token you control.)"""
koreni_besed(opis)

In [ ]:
razpredelnica_s_koreni_oracle_textov = podatkovna_baza_kart.oracle_text.apply(koreni_besed)
razpredelnica_s_koreni_oracle_textov_brez_korenov = podatkovna_baza_kart.oracle_text.apply(koreni_besed, ne_naredi_korenov = True)
print(razpredelnica_s_koreni_oracle_textov)

In [ ]:
koreni_tekstov = (
    razpredelnica_s_koreni_oracle_textov.unstack(  # vnose v razpredelnici razpotegnemo v eno vrsto (z gnezdenim indeksom)
    ).reset_index(                          # podatke iz indeksa pretvorimo v razpredelnico
    )[
        ["level_1", 0]                           # izberemo le relevantna dva stolpca
    ].rename(
        {"level_1": "indeks_oracle_teksta", 0: "koren"},
        axis = 1    # stolpcema damo smiselni imeni                            # s tem povemo, da preimenujemo stolpce in ne vrstic
    ).dropna(                               # odstranimo manjkajoče vnose
    )
)

koreni_tekstov_brez_korenov = (
    razpredelnica_s_koreni_oracle_textov_brez_korenov.unstack(  # vnose v razpredelnici razpotegnemo v eno vrsto (z gnezdenim indeksom)
    ).reset_index(                          # podatke iz indeksa pretvorimo v razpredelnico
    )[
        ["level_1", 0]                           # izberemo le relevantna dva stolpca
    ].rename(
        {"level_1": "indeks_oracle_teksta", 0: "koren"},
        axis = 1    # stolpcema damo smiselni imeni                            # s tem povemo, da preimenujemo stolpce in ne vrstic
    ).dropna(                               # odstranimo manjkajoče vnose
    )
)

koreni_tekstov

In [ ]:
#
tabela_barv = podatkovna_baza_kart[["barva"]]
tabela_barv.index.name = "indeks_oracle_teksta"
#print(koreni_tekstov, tabela_barv)

skupna_tabela = pd.merge(koreni_tekstov, tabela_barv, left_on="indeks_oracle_teksta", right_on="indeks_oracle_teksta")
skupna_tabela_brez_korenov = pd.merge(koreni_tekstov_brez_korenov, tabela_barv, left_on="indeks_oracle_teksta", right_on="indeks_oracle_teksta")
print(skupna_tabela)

In [ ]:
# Izkazalo se je, da je bolje, če imamo zelo majhen koeficient obtežitve!
koeficient_obtezitve_nevsebovanosti_korena = 0.0001
stolpec_verjetnosti_korenov_pri_oracle_tekstu = skupna_tabela.groupby(["barva", "koren"]).size() / pojavitve_barve
verjetnosti_korenov_pri_oracle_tekstu = stolpec_verjetnosti_korenov_pri_oracle_tekstu.unstack().transpose().fillna(koeficient_obtezitve_nevsebovanosti_korena / len(podatkovna_baza_kart))
stolpec_verjetnosti_korenov_pri_oracle_tekstu_brez_korenov = skupna_tabela_brez_korenov.groupby(["barva", "koren"]).size() / pojavitve_barve
verjetnosti_korenov_pri_oracle_tekstu_brez_korenov = stolpec_verjetnosti_korenov_pri_oracle_tekstu_brez_korenov.unstack().transpose().fillna(koeficient_obtezitve_nevsebovanosti_korena / len(podatkovna_baza_kart))
#verjetnosti_korenov_pri_oracle_tekstu
print(verjetnosti_korenov_pri_oracle_tekstu)

In [ ]:
def ugani_barvo(opis, ne_naredi_korenov = False):
    koreni_oracle_teksta = koreni_besed(opis, ne_naredi_korenov=ne_naredi_korenov)
    if ne_naredi_korenov:
        r = verjetnosti_korenov_pri_oracle_tekstu_brez_korenov.reset_index()
    else:
        r = verjetnosti_korenov_pri_oracle_tekstu.reset_index()
    verjetnosti = r[r.koren.isin(koreni_oracle_teksta)].product(numeric_only=True) * verjetnosti_barve
    return verjetnosti.sort_values(ascending=False).head(10)


poskusni_tekst_karte = """
Trample

Bioessence Hydra enters the battlefield with a +1/+1 counter on it for each loyalty counter on planeswalkers you control.

Whenever one or more loyalty counters are put on planeswalkers you control, put that many +1/+1 counters on Bioessence Hydra.
"""
print("Naivni Bayesov klasifikator, če ustvarimo korene:")
print(ugani_barvo(poskusni_tekst_karte), "\n")
print("Naivni Bayesov klasifikator, če ne ustvarimo korenov:")
print(ugani_barvo(poskusni_tekst_karte, ne_naredi_korenov=True))